In [ ]:
!pip install -qU uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 27.0 MB/s eta 0:00:00


In [ ]:
import numpy
numpy.__version__

'2.0.2'

In [ ]:
!uv pip install -q \
    "torch>=2.8.0" "triton>=3.4.0" numpy \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    torchvision bitsandbytes \
    git+https://github.com/huggingface/transformers \
    git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels

In [ ]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    "unsloth/gpt-oss-20b-unsloth-bnb-4bit", # 20B model using bitsandbytes 4bit quantization
    "unsloth/gpt-oss-120b-unsloth-bnb-4bit",
    "unsloth/gpt-oss-20b", # 20B model using MXFP4 format, OpenAI's MXFP4 Triton kernels combined with Unsloth's kernels!
    "unsloth/gpt-oss-120b",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b-unsloth-bnb-4bit",
    dtype = None, # None for auto detection
    max_seq_length = 4096,
    load_in_4bit = False,
    full_finetuning = False,
)

### Reasoning Effort
The `gpt-oss` models from OpenAI include a feature that allows users to adjust the model's "reasoning effort." This gives you control over the trade-off between the model's performance and its response speed (latency) which by the amount of token the model will use to think.

----

The `gpt-oss` models offer three distinct levels of reasoning effort you can choose from:

* **Low**: Optimized for tasks that need very fast responses and don't require complex, multi-step reasoning.
* **Medium**: A balance between performance and speed.
* **High**: Provides the strongest reasoning performance for tasks that require it, though this results in higher latency.

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low",
).to(model.device)

_ = model.generate(**inputs, max_new_tokens = 512, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-19

Reasoning: low

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>Equation: x^5 +3x^4 -10 =3 => x^5 +3x^4 -13 =0. Need solve. Likely numeric. Maybe integer root? try x=1:1+3-13=-9. x=2:32+48-13=67. So root between1 and2. Try x=1.2: (1.2^5=2.48832)+(3*1.2^4=3*2.0736=6.2208)-13= -4.29088. x=1.4: 1.4^5=5.37824 +3*1.4^4=3*3.8416=11.5248 sum=16.90304-13=3.90304. So between1.2 and1.4. Try1.3:1.3^5=3.71293+3*1.3^4=3*2.8561=8.5683 sum=12.28123-13=-0.71877. try1.33:1.33^5=4.099+3*1.33^4(1.33^4=3.141)=9.423 sum=13.522-13=0.522. try1.32:1.32^5=3.912+3*1.32^4(1.32^4=2.988)=8.964 sum=12.876-13=-0.124. try1.323: approximate root ~1.323. So answer. No other real roots? check negative large: x=-... function negative l

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "medium",
).to(model.device)

_ = model.generate(**inputs, max_new_tokens = 1024, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-19

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>The prompt: "Solve x^5 + 3x^4 - 10 = 3." So the equation: x^5 + 3x^4 - 10 = 3. That simplifies to x^5 + 3x^4 - 13 = 0. We need to find roots. It's a quintic polynomial: x^5 + 3x^4 - 13 = 0. Can we find rational roots? Potential rational roots: ±1, ±13. Try substituting x=1: 1+3-13=-9. x=-1: -1+3-13=-11. x=13: huge 13^5 +... huge positive minus 13. x = -13: huge negative minus 13. So no integer root. Maybe rational like ±13, ±1, ±13, ±... nothing else. Maybe root is something like 1? No. Might have one real root and others complex. Use numeric methods. Let's find approximate real root. Let f(x)=x^5+3x^4-13. The dominant term for large 

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "How many 'ss' in sstrawberiess"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low",
).to(model.device)

_ = model.generate(**inputs, max_new_tokens = 512, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-19

Reasoning: low

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>How many 'ss' in sstrawberiess<|end|><|start|>assistant<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-19

Reasoning: low

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>How many 'ss' in sstrawberiess<|end|><|start|>assistant<|channel|>analysis<|message|>Count 'ss' substrings. The string: s s t r a w b e r i e s s. Let's list: indices 0-1 'ss' at start. Also at end indices last two: 'ss'. Any in middle? No. So 2.<|end|><|start|>assistant<|channel|>final<|message|>There are **two** occurrences of the substring “ss” in “sstrawberiess”: one at the very beginning a

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Priya has 3 brothers and all brothers have two sisters each, how many sisters does Priya have?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high",
).to(model.device)

_ = model.generate(**inputs, max_new_tokens = 512, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-19

Reasoning: high

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>Priya has 3 brothers and all brothers have two sisters each, how many sisters does Priya have?<|end|><|start|>assistant<|channel|>analysis<|message|>We need to analyze: question: "Priya has 3 brothers and ... all brothers have two sisters each, how many sisters does Priya have?" We need to find number of sisters in this family. Let's think: There is Priya (female). She has 3 siblings who are brothers. Each brother has two sisters. Priya presumably is one of the sisters for each brother? Let's parse: "Priya has 3 brothers and all brothers have two sisters each." Typically this puzzle implies that the brothers each have 2 sisters total, which includes Priya and maybe another sister. We need to find number of sisters t

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Trinamool Congress leader Mahua Moitra has moved the Supreme Court against her expulsion from the Lok Sabha over the cash-for-query allegations against her. Moitra was ousted from the Parliament last week after the Ethics Committee of the Lok Sabha found her guilty of jeopardising national security by sharing her parliamentary portal's login credentials with businessman Darshan Hiranandani. ### You are an NER engine. Please reply in json with all the entities and relevant names."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high",
).to(model.device)

_ = model.generate(**inputs, max_new_tokens = 1024, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-19

Reasoning: high

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>Trinamool Congress leader Mahua Moitra has moved the Supreme Court against her expulsion from the Lok Sabha over the cash-for-query allegations against her. Moitra was ousted from the Parliament last week after the Ethics Committee of the Lok Sabha found her guilty of jeopardising national security by sharing her parliamentary portal's login credentials with businessman Darshan Hiranandani. ### You are an NER engine. Please reply in json with all the entities and relevant names.<|end|><|start|>assistant<|channel|>analysis<|message|>The user wants NER of the given paragraph. They want a JSON with all entities and relevant names. So I need to identify entities: People, organizations, places, etc. Then include relevant